# Hallucination as Signal Decay: Colab-Ready Walkthrough

Welcome! This notebook accompanies our paper **"When Models Know but Don’t Say"**. 

It is designed to be **completely standalone**. You can run this on Google Colab (using a T4 or A100 GPU) to verify our findings.

### What we'll do:
1.  **Reproduce the "Knowledge Gap"**: See the massive difference between what the model *knows* (Pairwise) and what it *says* (Generation).
2.  **Trace the Signal**: Use the **Logit Lens** to watch the truth decay in real-time.
3.  **Fix it**: Apply **Logit Mixing** to recover the correct answer.

> **Note**: This notebook uses `meta-llama/Llama-3.1-8B`. You need access to this model on HuggingFace and a User Access Token.


In [ ]:
!pip install transformer_lens circuitsvis datasets accelerate protobuf sentencepiece

## 0. Authentication
Llama-3.1 is a gated model. Please provide your HuggingFace token below.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## 1. Setup & Load Model
We use `TransformerLens` to hook into the model's internal activations.


In [ ]:
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformer_lens import HookedTransformer

sns.set_context("notebook", font_scale=1.2)
sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'sans-serif'

# Config
MODEL_NAME = "meta-llama/Llama-3.1-8B"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading {MODEL_NAME} on {DEVICE}...")

# Load model in 4-bit/8-bit if on Colab (optional, standard load here)
# If you run out of memory on Colab T4, try:
# model = HookedTransformer.from_pretrained(MODEL_NAME, device=DEVICE, load_in_8bit=True)

model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    device=DEVICE,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False
)

print("Model loaded!")

## 2. The Knowledge Gap

We examine **CounterFact** examples.
*   **Pairwise**: Does the model rank True > False?
*   **Generation**: Does the model output True?


In [ ]:
from datasets import load_dataset

# Load a tiny subset for demo purposes
dataset = load_dataset("NeelNanda/counterfact-tracing", split="train[:10]")

print(f"Checking {len(dataset)} examples...")

results = []

for item in dataset:
    prompt = item['prompt']
    target_true = item['target_true'].strip()
    target_false = item['target_false'].strip()
    
    # 1. Run Model
    # prepend_bos=True is crucial for Llama
    logits = model(prompt, prepend_bos=True)
    last_token_logits = logits[0, -1, :]
    
    # 2. Check Generation (Top-1)
    pred_token_id = torch.argmax(last_token_logits).item()
    pred_token = model.to_string(pred_token_id)
    is_correct_gen = (pred_token.strip().lower() == target_true.lower())
    
    # 3. Check Pairwise (True vs False)
    # We need to prepend a space to match Llama tokenization usually
    true_token_id = model.to_single_token(f" {target_true}")
    false_token_id = model.to_single_token(f" {target_false}")
    
    true_logit = last_token_logits[true_token_id].item()
    false_logit = last_token_logits[false_token_id].item()
    is_correct_pair = (true_logit > false_logit)
    
    results.append({
        "Prompt": prompt,
        "True": target_true,
        "Pred": pred_token,
        "Pairwise": "✅" if is_correct_pair else "❌",
        "Gen": "✅" if is_correct_gen else "❌"
    })

df_gap = pd.DataFrame(results)
display(df_gap)

# Filter for the phenomenon: Pairwise Correct but Generation Wrong
gap = df_gap[df_gap['Pairwise'] == '✅'][df_gap['Gen'] == '❌']
print(f"\nFound {len(gap)} examples where the model KNOWS but FAILS to generate.")


## 3. Investigating "Signal Decay"

Let's pick an error case and trace the probability of the correct token across all layers.


In [ ]:
if len(gap) > 0:
    example = gap.iloc[0]
else:
    example = df_gap.iloc[0] # Fallback

prompt = example['Prompt']
target = example['True']

print(f"Analyzing Prompt: '{prompt}'")
print(f"Target: '{target}'")

# Run with cache to get internals
logits, cache = model.run_with_cache(prompt)
target_id = model.to_single_token(f" {target}")

layer_probs = []

for layer in range(model.cfg.n_layers):
    # Retrieve residual stream at the last token position
    resid = cache[f"blocks.{layer}.hook_resid_post"][0, -1, :]
    
    # Apply LayerNorm and Unembedding (Logit Lens)
    # Note: HookedTransformer makes this easy
    normalized = model.ln_final(resid)
    layer_logits = model.unembed(normalized)
    
    # Get Probability
    probs = torch.softmax(layer_logits, dim=-1)
    target_prob = probs[target_id].item()
    layer_probs.append(target_prob)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(layer_probs, marker='o', linewidth=2.5, color='#d62728', label=f"P({target})")
plt.title(f"Signal Decay Trace: '{target}'")
plt.xlabel("Layer")
plt.ylabel("Probability")
plt.axhline(0, color='gray', linestyle='--')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Observation: The probability likely peaks around layer 15-20 and then drops.")

## 4. The Fix: Logit Mixing

We identified that the signal exists in the middle. Let's use it!
We will intervene by adding a fraction of the mid-layer logits to the final logits.

$$ L_{final} \leftarrow (1 - \alpha) L_{final} + \alpha L_{mid} $$


In [ ]:
alpha = 0.2
mid_layer = 20

# 1. Get Mid-Layer Logits
resid_mid = cache[f"blocks.{mid_layer}.hook_resid_post"][0, -1, :]
norm_mid = model.ln_final(resid_mid)
logits_mid = model.unembed(norm_mid)

# 2. Get Final Logits
logits_final = logits[0, -1, :]

# 3. Mix
logits_mixed = (1 - alpha) * logits_final + (alpha) * logits_mid

# 4. Check Result
new_pred_id = torch.argmax(logits_mixed).item()
new_pred = model.to_string(new_pred_id)

print(f"Original Prediction: '{example['Pred']}'")
print(f"Intervention (alpha={alpha}): '{new_pred}'")

if new_pred.strip().lower() == target.lower():
    print("🚀 SUCCESS: Knowledge recovered!")
else:
    print("Failed. Try tuning alpha or layer.")